# PREPARE DATA

In [35]:
# Import libraries
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt


def is_time_between(start_hour, end_hour, check_hour):
    if start_hour < end_hour:
        return start_hour <= check_hour <= end_hour
    else:  # crosses midnight
        return check_hour >= start_hour or check_hour <= end_hour
    
def is_time_between_exc(start_hour, end_hour, check_hour):
    if start_hour < end_hour:
        return start_hour >= check_hour < end_hour
    else:  # crosses midnight
        return check_hour >= start_hour or check_hour < end_hour

dataset_name = "GBP_USD_M15"
# Read the ohlcv data
# oanda_path = "/projects/genomic-ml/da2343/ml_project_2/data/gen_oanda_data"
oanda_path = "/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data"
df = pd.read_csv(f'{oanda_path}/{dataset_name}_processed_data.csv')
df = df.rename(columns={"time": "Time"})
df["Index"] = df.index
y = df[["Close"]]
offset = y.index[0]
df["Time"] = pd.to_datetime(df["Time"])
df['EMA_100'] = ta.EMA(df['Close'], timeperiod=100)
# drop na values
df = df.dropna()
# filter when time is only in 2016
df = df[df["Time"] >= "2010-01-01 00:00:00"]
df

,Time,Open,High,Low,Close,Volume,SMA_20,SMA_30,SMA_50,SMA_100,...,WILLR,BBANDS_Upper,BBANDS_Middle,BBANDS_Lower,AD,ADOSC,VOLUME_RSI,MFI,Index,EMA_100
127974,2010-01-03 18:00:00+00:00,1.61592,1.61592,1.61578,1.61592,3,1.616430,1.616275,1.617253,1.613037,...,-63.138686,1.617700,1.616102,1.614504,-301469.081496,160.158436,43.875523,25.225853,127974,1.612496
127975,2010-01-03 18:15:00+00:00,1.61601,1.61685,1.61567,1.61635,7,1.616459,1.616254,1.617340,1.613125,...,-47.445255,1.617749,1.616138,1.614527,-301468.013699,137.919552,44.060038,26.393861,127975,1.612572
127976,2010-01-03 18:30:00+00:00,1.61560,1.61635,1.61535,1.61535,11,1.616489,1.616288,1.617388,1.613201,...,-83.941606,1.617718,1.616164,1.614610,-301479.013699,112.783686,44.257395,26.299953,127976,1.612627
127977,2010-01-03 18:45:00+00:00,1.61530,1.61568,1.61514,1.61561,25,1.616505,1.616304,1.617417,1.613284,...,-74.452555,1.616453,1.615772,1.615091,-301460.495180,98.140025,44.988927,20.526937,127977,1.612686
127978,2010-01-03 19:00:00+00:00,1.61550,1.61563,1.61536,1.61544,22,1.616456,1.616320,1.617419,1.613364,...,-80.656934,1.616462,1.615734,1.615006,-301469.458143,80.375764,44.853091,21.489753,127978,1.612741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482027,2024-02-16 09:00:00+00:00,1.25766,1.25846,1.25746,1.25805,1086,1.258325,1.258349,1.258823,1.257986,...,-81.677019,1.258510,1.257960,1.257410,-296385.500672,-715.953387,51.372062,48.919234,482027,1.258167
482028,2024-02-16 09:15:00+00:00,1.25806,1.25864,1.25786,1.25854,1180,1.258358,1.258339,1.258800,1.258020,...,-66.459627,1.258718,1.258018,1.257318,-295508.064775,-328.049816,53.263920,54.462298,482028,1.258175
482029,2024-02-16 09:30:00+00:00,1.25856,1.25890,1.25842,1.25850,1455,1.258381,1.258332,1.258773,1.258056,...,-67.701863,1.258841,1.258196,1.257551,-296478.064775,-448.175644,58.366977,56.839208,482029,1.258181
482030,2024-02-16 09:45:00+00:00,1.25850,1.25882,1.25831,1.25858,1198,1.258399,1.258330,1.258752,1.258089,...,-65.217391,1.258983,1.258266,1.257549,-296407.594187,-434.152328,52.588013,55.626229,482030,1.258189


## STRATEGY ADAPTIVE PROFIT

In [ ]:
# Initialize the variables for position, entry price and stop loss
position = 0
entry_price = 0
stop_loss = 0
trades = []

for i, row in df.iterrows():
    current_time = row["Time"]
    condition_one = (
            (row["MACD_Crossover_Change"] > 0)
            and (row["Close"] > row["EMA_100"])
            and is_time_between(start_hour, end_hour, current_time.hour)
    )
    condition_two = (
            (row["MACD_Crossover_Change"] < 0)
            and (row["Close"] < row["EMA_100"])
            and is_time_between(start_hour, end_hour, current_time.hour)
    )
    if position == 0:
        if condition_one:
            position = 1
            entry_price = row['Close']
            stop_loss = entry_price - atr_delta*row['ATR']
            init_stop_loss = stop_loss
            trades.append(['Buy', current_time, entry_price, 0])
        elif condition_two:
            position = -1 
            entry_price = row['Close']
            stop_loss = entry_price + atr_delta*row['ATR']
            init_stop_loss = stop_loss
            trades.append(['Sell', current_time, entry_price, 0])
    elif position == 1:
        if row['Low'] < stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (exit_price - entry_price) 
            target_loss = (entry_price - init_stop_loss)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Buy', current_time, exit_price, risk_reward]
        else:
            stop_loss = max(stop_loss,  row['Close'] - atr_delta*row['ATR'])
    elif position == -1:
        if row['High'] > stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (entry_price - exit_price)
            target_loss = (init_stop_loss - entry_price)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Sell', current_time, exit_price, risk_reward]
        else:
            stop_loss = min(stop_loss, row['Close'] + atr_delta*row['ATR'])

# Convert the list of trades to a dataframe
trades_df = pd.DataFrame(trades, columns=['Action', 'Time', 'Price', 'PnL'])

# Print the trades
# Add the PnL_label column where 1 is a positive or breakeven return and 0 is a negative return
trades_df['PnL_label'] = np.where(trades_df['PnL'] >= 0, 1, 0)
trades_df

In [ ]:
trades_df['Time'] = pd.to_datetime(trades_df['Time'])
trades_df['Year'] = trades_df['Time'].dt.year
trades_df['Month'] = trades_df['Time'].dt.month
sub_trades_df = trades_df[trades_df['Year'] == 2016].copy()
sub_trades_df = sub_trades_df[['Month', f'PnL']]
sub_trades_df = sub_trades_df.groupby(['Month']).agg({f'PnL': 'sum'}).reset_index()

# using the label column find the cumulative sum
sub_trades_df["cumulative_sum"] = sub_trades_df["PnL"].cumsum()
#plot first 100 trades
sub_trades_df["cumulative_sum"].plot()
# sub_trades_df

In [ ]:
# EUR_USD_H1 : off-years = 2014, 2016, 2021
# EUR_JPY_H1 : off-years = 2016
# GBP_JPY_H1 : off-years = 2023
# USD_MXN_H1 : off-years = 2018, 2019, 2022
# NZD_JPY_H1 : off-years = 2014, 2015, 2019

# AUD_HKD_H1 : off-years = 2015, 2018, 2019, 2021
# EUR_GBP_H1 : off-years = 2018, 2021, 2022, 2023

# ZAR_JPY_H1 : off-years = 2014, 2015, 2017, 2019
# USD_NOK_H1 : off-years = 2014, 2015
# USD_CHF_H1 : off-years = 2014
# TRY_JPY_H1 : off-years = 2014, 2019, 2021, 2022, 2023
# AUD_USD_H1 : off-years = 2015,2016,2018, 2019, 2022

In [ ]:
trades_df["PnL_label"].mean()

## STRATEGY FIXED PROFIT AND LOSS

In [ ]:
# Initialize the variables for position, entry price and stop loss
position = 0
entry_price = 0
stop_loss = 0
trades = []
atr_delta = 1.5
start_hour = 13
end_hour = 17

for i, row in df.iterrows():
    current_time = row["Time"]
    condition_buy = (
            (row["MACD_Crossover_Change"] > 0)
            and (row["Close"] > row["EMA_100"])
            and is_time_between(start_hour, end_hour, current_time.hour)
    )
    condition_sell = (
            (row["MACD_Crossover_Change"] < 0)
            and (row["Close"] < row["EMA_100"])
            and is_time_between(start_hour, end_hour, current_time.hour)
    )
    if position == 0:
        if condition_buy:
            position = 1
            entry_price = row['Close']
            stop_loss = entry_price - atr_delta*row['ATR']
            take_profit = entry_price + atr_delta*row['ATR']
            init_stop_loss = stop_loss
            trades.append(['Buy', current_time, entry_price, 0])
        elif condition_sell:
            position = -1 
            entry_price = row['Close']
            stop_loss = entry_price + atr_delta*row['ATR']
            take_profit = entry_price - atr_delta*row['ATR']
            init_stop_loss = stop_loss
            trades.append(['Sell', current_time, entry_price, 0])
    elif position == 1:
        if row['Low'] < stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (exit_price - entry_price) 
            target_loss = (entry_price - init_stop_loss)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Buy', current_time, exit_price, risk_reward]
        elif row['High'] > take_profit:
            position = 0
            exit_price = take_profit
            target_profit = (exit_price - entry_price) 
            target_loss = (entry_price - init_stop_loss)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Buy', current_time, exit_price, risk_reward]
        else:
            pass
    elif position == -1:
        if row['High'] > stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (entry_price - exit_price)
            target_loss = (init_stop_loss - entry_price)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Sell', current_time, exit_price, risk_reward]
        elif row['Low'] < take_profit:
            position = 0
            exit_price = take_profit
            target_profit = (entry_price - exit_price)
            target_loss = (init_stop_loss - entry_price)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Sell', current_time, exit_price, risk_reward]    
        else:
            pass

# Convert the list of trades to a dataframe
trades_df = pd.DataFrame(trades, columns=['Action', 'Time', 'Price', 'PnL'])

# Print the trades
# Add the PnL_label column where 1 is a positive or breakeven return and 0 is a negative return
trades_df['PnL_label'] = np.where(trades_df['PnL'] >= 0, 1, 0)
trades_df

In [ ]:
trades_df['Time'] = pd.to_datetime(trades_df['Time'])
trades_df['Year'] = trades_df['Time'].dt.year
trades_df['Month'] = trades_df['Time'].dt.month
sub_trades_df = trades_df[trades_df['Year'] == 2016].copy()
sub_trades_df = sub_trades_df[['Month', f'PnL']]
sub_trades_df = sub_trades_df.groupby(['Month']).agg({f'PnL': 'sum'}).reset_index()

# using the label column find the cumulative sum
sub_trades_df["cumulative_sum"] = sub_trades_df["PnL"].cumsum()
#plot first 100 trades
sub_trades_df["cumulative_sum"].plot()
# sub_trades_df

In [ ]:
# AUD_HKD_H1 : off-years = 2015, 2018
# EUR_GBP_H1: 2018, 2019, 2021


## STRATEGY FIXED PROFIT AND ADAPTIVE LOSS

In [ ]:
# Initialize the variables for position, entry price and stop loss
position = 0
entry_price = 0
stop_loss = 0
trades = []

for i, row in df.iterrows():
    current_time = row["Time"]
    condition_buy = (
            (row["MACD_Crossover_Change"] > 0)
            and (row["Close"] > row["EMA_100"])
            and is_time_between(start_hour, end_hour, current_time.hour)
    )
    condition_sell = (
            (row["MACD_Crossover_Change"] < 0)
            and (row["Close"] < row["EMA_100"])
            and is_time_between(start_hour, end_hour, current_time.hour)
    )
    if position == 0:
        if condition_buy:
            position = 1
            entry_price = row['Close']
            stop_loss = entry_price - atr_delta*row['ATR']
            take_profit = entry_price + atr_delta*row['ATR']
            init_stop_loss = stop_loss
            trades.append(['Buy', current_time, entry_price, 0])
        elif condition_sell:
            position = -1 
            entry_price = row['Close']
            stop_loss = entry_price + atr_delta*row['ATR']
            take_profit = entry_price - atr_delta*row['ATR']
            init_stop_loss = stop_loss
            trades.append(['Sell', current_time, entry_price, 0])
    elif position == 1:
        if row['Low'] < stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (exit_price - entry_price) 
            target_loss = (entry_price - init_stop_loss)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Buy', current_time, exit_price, risk_reward, take_profit, stop_loss]
        elif row['High'] > take_profit:
            position = 0
            exit_price = take_profit
            target_profit = (exit_price - entry_price) 
            target_loss = (entry_price - init_stop_loss)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Buy', current_time, exit_price, risk_reward, take_profit, stop_loss]
        else:
            # stop loss must not be greater than the take profit
            # pass
            next_stop_loss = row['Close'] - atr_delta*row['ATR']
            if next_stop_loss < take_profit:
                stop_loss = max(stop_loss, next_stop_loss)
    elif position == -1:
        if row['High'] > stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (entry_price - exit_price)
            target_loss = (init_stop_loss - entry_price)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Sell', current_time, exit_price, risk_reward, take_profit, stop_loss]
        elif row['Low'] < take_profit:
            position = 0
            exit_price = take_profit
            target_profit = (entry_price - exit_price)
            target_loss = (init_stop_loss - entry_price)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Sell', current_time, exit_price, risk_reward, take_profit, stop_loss] 
        else:
            # pass
            next_stop_loss = row['Close'] + atr_delta*row['ATR']
            if next_stop_loss > take_profit:
                stop_loss = min(stop_loss, next_stop_loss)

# Convert the list of trades to a dataframe
trades_df = pd.DataFrame(trades, columns=['Action', 'Time', 'Price', 'PnL', 'Take_Profit', 'Stop_Loss'])

# Print the trades
# Add the PnL_label column where 1 is a positive or breakeven return and 0 is a negative return
trades_df['PnL_label'] = np.where(trades_df['PnL'] >= 0, 1, 0)
trades_df

In [ ]:
trades_df["cumulative_sum"] = trades_df["PnL"].cumsum()
trades_df["cumulative_sum"].plot()

In [ ]:
trades_df['Time'] = pd.to_datetime(trades_df['Time'])
trades_df['Year'] = trades_df['Time'].dt.year
trades_df['Month'] = trades_df['Time'].dt.month
sub_trades_df = trades_df[trades_df['Year'] == 2014].copy()
sub_trades_df = sub_trades_df[['Month', f'PnL']]
sub_trades_df = sub_trades_df.groupby(['Month']).agg({f'PnL': 'sum'}).reset_index()
sub_trades_df["cumulative_sum"] = sub_trades_df["PnL"].cumsum()
sub_trades_df["cumulative_sum"].plot()

In [ ]:
x = 642
y = 599

p = (x) / (x+y)
p * 100

In [ ]:
df

## OPENING RANGE BREAKOUT STRATEGY

In [36]:
# Initialize the variables for position, entry price and stop loss
position = 0
entry_price = 0
stop_loss = 0
trades = []


atr_delta = 3
start_hour = 8
end_hour = 17
range_start_hour = 5

for i, row in df.iterrows():
    current_time = row["Time"]
    latest_trade_date = trades[-1][1].date() if trades else None
    current_date = current_time.date()

    if (
        position == 0
        and is_time_between_exc(start_hour, end_hour, current_time.hour)
        and current_date != latest_trade_date
    ):
        # range_df should be from range_start_hour to range_end_hour
        range_df = df[
            (df["Time"].dt.hour >= range_start_hour)
            & (df["Time"].dt.hour < start_hour)
            & (df["Time"].dt.date == current_date)
        ]

        if range_df.empty:
            continue

        opening_high = range_df["High"].max()
        opening_low = range_df["Low"].min()
        opening_range = opening_high - opening_low
        
        if opening_range < row["ATR"]:
            continue

        condition_buy = (row["Close"] > opening_high)
        condition_sell = (row["Close"] < opening_low)

        if condition_buy:
            position = 1
            entry_price = row['Close']
            # stop_loss = entry_price - atr_delta*row['ATR']
            # take_profit = entry_price + atr_delta*row['ATR']

            take_profit = entry_price + opening_range
            stop_loss = entry_price - 0.5*opening_range
            init_stop_loss = stop_loss
            trades.append(['Buy', current_time, entry_price, 0])
        elif condition_sell:
            position = -1 
            entry_price = row['Close']
            # stop_loss = entry_price + atr_delta*row['ATR']
            # take_profit = entry_price - atr_delta*row['ATR']
            
            stop_loss = entry_price + 0.5*opening_range
            take_profit = entry_price - opening_range
            init_stop_loss = stop_loss
            trades.append(['Sell', current_time, entry_price, 0])
    elif position == 1:
        if row['Low'] < stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (exit_price - entry_price) 
            target_loss = (entry_price - init_stop_loss)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Buy', current_time, exit_price, risk_reward]
        elif row['High'] > take_profit:
            position = 0
            exit_price = take_profit
            target_profit = (exit_price - entry_price) 
            target_loss = (entry_price - init_stop_loss)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Buy', current_time, exit_price, risk_reward]
        else:
            pass
    elif position == -1:
        if row['High'] > stop_loss:
            position = 0
            exit_price = stop_loss
            target_profit = (entry_price - exit_price)
            target_loss = (init_stop_loss - entry_price)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Sell', current_time, exit_price, risk_reward]
        elif row['Low'] < take_profit:
            position = 0
            exit_price = take_profit
            target_profit = (entry_price - exit_price)
            target_loss = (init_stop_loss - entry_price)
            risk_reward = (target_profit/target_loss)
            trades[-1] = ['Sell', current_time, exit_price, risk_reward]    
        else:
            pass


In [ ]:
# Convert the list of trades to a dataframe
trades_df = pd.DataFrame(trades, columns=['Action', 'Time', 'Price', 'PnL'])

# Print the trades
# Add the PnL_label column where 1 is a positive or breakeven return and 0 is a negative return
trades_df['PnL_label'] = np.where(trades_df['PnL'] >= 0, 1, 0)
trades_df["PnL_label"].mean() 

In [ ]:
trades_df["cumulative_sum"] = trades_df["PnL"].cumsum()
trades_df["cumulative_sum"].plot()

In [ ]:
trades_df